In [2]:
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import select_features
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import roll_time_series

import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

C:\Users\TEMP\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Формируем выборку временного ряда

In [3]:
df=pd.read_pickle('MG_Sales.pickle',compression='gzip')

#&(df['ТоварЦеноваяГруппа']=='0-2100')
begin_period=dt.datetime(2015,1,1)
prediction_period=dt.datetime(2017,1,1)

df.drop(df[df['Дата']<begin_period].index, inplace=True)
df.index.name='index'
y=df['Количество']
df.drop('Количество',inplace=True, axis=1)

#переводим номинальные данные
categorical_columns = [c for c in df.columns if df[c].dtype.name == 'object']
onehot_encoder =  LabelEncoder()
for col in categorical_columns:
    df[col] = onehot_encoder.fit_transform(df[col]).astype('int64')

#time_series=#pd.DataFrame(data=df[(df['Дата']>=begin_period)].groupby('Дата')['Количество'].sum())
#time_series.index.name='ds'
#time_series.columns=['y']
#time_series['val']=0
#time_series=time_series.reset_index().reset_index()
#y=time_series.y
#time_series.drop('y',inplace=True, axis=1)

In [ ]:
features_filtered_direct = extract_relevant_features(df.reset_index(), y,column_id='index',
                                                      column_sort='Дата')
features_filtered_direct

Feature Extraction:   0%|                                | 0/8 [00:00<?, ?it/s]

## Формируем выборку

In [ ]:
#Фиксируем аномально низкие и высокие продажи
ul=5#Персентиль высоких продаж 2
ll=5#7#Персентиль низких продаж 10
md=3#ширина медианы

ulim=np.percentile(time_series['y'], 100.-ul)
llim=np.percentile(time_series['y'], ll)
med=np.percentile(time_series['y'], [50-md,50+md])


time_series['Квантили']=0
time_series.loc[time_series.y<med[0],'Квантили']=-1
time_series.loc[time_series.y>med[1],'Квантили']=1
time_series.loc[time_series.y<llim,'Квантили']=-2
time_series.loc[time_series.y>ulim,'Квантили']=2

#вычисляем год назад
def yearsago(years, from_date):
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:        
        return from_date.replace(month=2, day=28,
                                 year=from_date.year-years)

#временные характеристики
def setNewValues(time_series):
    time_series['День недели'] = time_series.index.weekday
    time_series['Месяц'] = time_series.index.month
    time_series['Год'] = time_series.index.year
    time_series['День месяца'] = time_series.index.day
    time_series['Выходные'] = time_series.index.weekday.isin([5,6])*1
    time_series['Праздник'] =df.groupby('Дата')['Праздник'].max()
    return time_series

def weekseason(time_series):
    time_series['Недельная сезонность']=time_series['День недели'].map(lambda cell: week_d.loc[cell,'Недельная сезонность'])
    return time_series

time_series=setNewValues(time_series)

#порядок дней в сезонности недельной продажи за исключением аномалий
week_d=pd.DataFrame(data=time_series[time_series['Квантили']==0].groupby('День недели')['y'].sum().sort_values())
week_d.insert(0,'Недельная сезонность',list(range(week_d.shape[0])))
for i in list(set(range(7))-set(week_d.index.values)):
    week_d.loc[i,'Недельная сезонность']=-1
    
time_series=weekseason(time_series)

#Вычленяем целевую переменную
y=time_series.y
time_series=time_series.drop(['y'], axis=1).reset_index()
time_series=time_series.reset_index()

In [ ]:
features_filtered_direct = extract_relevant_features(time_series, y.reset_index().reset_index().drop('ds',axis=1).values,
                                                     column_id='index', column_sort='ds',parallelization='per_sample')

In [ ]:
tdf=roll_time_series(time_series,column_id='index', column_sort='ds',rolling_direction=365,column_kind =None)

In [ ]:
tdf.tail()

In [ ]:
df.index.name='index'

In [ ]:
df.head(5)

In [ ]:
OneHotEncoder(df['Коллекция'])

In [ ]:
onehot_encoder.fit_transform(df['Коллекция'])